In [54]:
import requests
import json
import pandas as pd
import os
import time
import re
import ast

In [2]:
games = pd.read_csv('../data/rawg_games_raw_data.csv')
developers = pd.read_csv('../data/rawg_developer_raw_data.csv')

## Cleaning Games DF

In [4]:
games

,Unnamed: 0,id,slug,name,released,tba,background_image,rating,rating_top,ratings,...,dominant_color,platforms,parent_platforms,genres,stores,clip,tags,esrb_rating,short_screenshots,community_rating
0,0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,https://media.rawg.io/media/games/20a/20aa03a1...,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 42...",...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 290376, 'store': {'id': 1, 'name': 'St...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
1,1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,https://media.rawg.io/media/games/618/618c2031...,4.64,5,"[{'id': 5, 'title': 'exceptional', 'count': 53...",...,0f0f0f,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 354780, 'store': {'id': 5, 'name': 'GO...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
2,2,4200,portal-2,Portal 2,2011-04-18,False,https://media.rawg.io/media/games/2ba/2bac0e87...,4.59,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",...,0f0f0f,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 465889, 'store': {'id': 2, 'name': 'Xb...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
3,3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,https://media.rawg.io/media/games/736/73619bd3...,3.57,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 4619, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
4,4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,https://media.rawg.io/media/games/021/021c4e21...,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 24...",...,0f0f0f,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 33824, 'store': {'id': 7, 'name': 'Xbo...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,21970,crimson-earth,Crimson Earth,2017-05-09,False,https://media.rawg.io/media/screenshots/507/50...,0.00,0,"[{'id': 1, 'title': 'skip', 'count': 2, 'perce...",...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 24731, 'store': {'id': 1, 'name': 'Ste...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,"[{'id': -1, 'image': 'https://media.rawg.io/me...",0.0
19996,19996,21553,slamit-pinball-big-score,SlamIt Pinball Big Score,2009-02-23,False,https://media.rawg.io/media/screenshots/ad7/ad...,0.00,0,"[{'id': 4, 'title': 'recommended', 'count': 1,...",...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 11, 'name': 'Arcade', 'slug': 'arcade'...","[{'id': 24251, 'store': {'id': 1, 'name': 'Ste...",NaN,"[{'id': 31, 'n

In [72]:
def remove_string(s):
    if re.search(r'\[.*?\]', s):
        list_str = re.search(r'\[.*?\]', s).group()
        extracted_list = ast.literal_eval(list_str)
        return extracted_list

In [76]:
remove_string(games['stores'][0])

[{'id': 290376,
  'store': {'id': 1,
   'name': 'Steam',
   'slug': 'steam',
   'domain': 'store.steampowered.com',
   'games_count': 113018,
   'image_background': 'https://media.rawg.io/media/games/2ba/2bac0e87cf45e5b508f227d281c9252a.jpg'}},
 {'id': 290375,
  'store': {'id': 3,
   'name': 'PlayStation Store',
   'slug': 'playstation-store',
   'domain': 'store.playstation.com',
   'games_count': 8051,
   'image_background': 'https://media.rawg.io/media/games/4cf/4cfc6b7f1850590a4634b08bfab308ab.jpg'}},
 {'id': 438095,
  'store': {'id': 11,
   'name': 'Epic Games',
   'slug': 'epic-games',
   'domain': 'epicgames.com',
   'games_count': 1400,
   'image_background': 'https://media.rawg.io/media/games/5c0/5c0dd63002cb23f804aab327d40ef119.jpg'}},
 {'id': 290377,
  'store': {'id': 7,
   'name': 'Xbox 360 Store',
   'slug': 'xbox360',
   'domain': 'marketplace.xbox.com',
   'games_count': 1915,
   'image_background': 'https://media.rawg.io/media/games/021/021c4e21a1824d2526f925eff6324653.

In [50]:
games['tags'][0]

"[{'id': 31, 'name': 'Singleplayer', 'slug': 'singleplayer', 'language': 'eng', 'games_count': 241108, 'image_background': 'https://media.rawg.io/media/games/b45/b45575f34285f2c4479c9a5f719d972e.jpg'}, {'id': 40847, 'name': 'Steam Achievements', 'slug': 'steam-achievements', 'language': 'eng', 'games_count': 46424, 'image_background': 'https://media.rawg.io/media/games/9fa/9fa63622543e5d4f6d99aa9d73b043de.jpg'}, {'id': 7, 'name': 'Multiplayer', 'slug': 'multiplayer', 'language': 'eng', 'games_count': 40923, 'image_background': 'https://media.rawg.io/media/games/021/021c4e21a1824d2526f925eff6324653.jpg'}, {'id': 40836, 'name': 'Full controller support', 'slug': 'full-controller-support', 'language': 'eng', 'games_count': 21864, 'image_background': 'https://media.rawg.io/media/games/f46/f466571d536f2e3ea9e815ad17177501.jpg'}, {'id': 13, 'name': 'Atmospheric', 'slug': 'atmospheric', 'language': 'eng', 'games_count': 37360, 'image_background': 'https://media.rawg.io/media/games/6cd/6cd653e

In [7]:
games.columns

Index(['Unnamed: 0', 'id', 'slug', 'name', 'released', 'tba',
       'background_image', 'rating', 'rating_top', 'ratings', 'ratings_count',
       'reviews_text_count', 'added', 'added_by_status', 'metacritic',
       'playtime', 'suggestions_count', 'updated', 'user_game',
       'reviews_count', 'saturated_color', 'dominant_color', 'platforms',
       'parent_platforms', 'genres', 'stores', 'clip', 'tags', 'esrb_rating',
       'short_screenshots', 'community_rating'],
      dtype='object')

In [8]:
games = games.drop(columns = ['background_image', 'saturated_color', 'dominant_color', 'short_screenshots'])

In [9]:
games['platforms'][0]

"[{'platform': {'id': 4, 'name': 'PC', 'slug': 'pc', 'image': None, 'year_end': None, 'year_start': None, 'games_count': 550071, 'image_background': 'https://media.rawg.io/media/games/4be/4be6a6ad0364751a96229c56bf69be59.jpg'}, 'released_at': '2013-09-17', 'requirements_en': {'minimum': 'Minimum:OS: Windows 10 64 Bit, Windows 8.1 64 Bit, Windows 8 64 Bit, Windows 7 64 Bit Service Pack 1, Windows Vista 64 Bit Service Pack 2* (*NVIDIA video card recommended if running Vista OS)Processor: Intel Core 2 Quad CPU Q6600 @ 2.40GHz (4 CPUs) / AMD Phenom 9850 Quad-Core Processor (4 CPUs) @ 2.5GHzMemory: 4 GB RAMGraphics: NVIDIA 9800 GT 1GB / AMD HD 4870 1GB (DX 10, 10.1, 11)Storage: 72 GB available spaceSound Card: 100% DirectX 10 compatibleAdditional Notes: Over time downloadable content and programming changes will change the system requirements for this game.  Please refer to your hardware manufacturer and www.rockstargames.com/support for current compatibility information. Some system compon

In [10]:
games['ratings'][0]

"[{'id': 5, 'title': 'exceptional', 'count': 4267, 'percent': 59.0}, {'id': 4, 'title': 'recommended', 'count': 2365, 'percent': 32.7}, {'id': 3, 'title': 'meh', 'count': 460, 'percent': 6.36}, {'id': 1, 'title': 'skip', 'count': 140, 'percent': 1.94}]"

## Cleaning Developers

In [12]:
developers

,Unnamed: 0,id,name,slug,games_count,image_background,games
0,0,1612,Valve Software,valve-software,44,https://media.rawg.io/media/games/46d/46d98e69...,"[{'id': 4200, 'slug': 'portal-2', 'name': 'Por..."
1,1,405,Ubisoft,ubisoft,346,https://media.rawg.io/media/games/2ee/2eef5ed5...,"[{'id': 4161, 'slug': 'far-cry-3', 'name': 'Fa..."
2,2,18893,Feral Interactive,feral-interactive,107,https://media.rawg.io/media/screenshots/375/37...,"[{'id': 7689, 'slug': 'rise-of-the-tomb-raider..."
3,3,3709,Ubisoft Montreal,ubisoft-montreal,121,https://media.rawg.io/media/games/99b/99b39612...,"[{'id': 4161, 'slug': 'far-cry-3', 'name': 'Fa..."
4,4,4132,Square Enix,square-enix,368,https://media.rawg.io/media/games/214/2140885d...,"[{'id': 10035, 'slug': 'hitman', 'name': 'Hitm..."
...,...,...,...,...,...,...,...
14995,14995,106339,JiSheng Fu,jisheng-fu,3,https://media.rawg.io/media/screenshots/f2f/f2...,"[{'id': 44219, 'slug': 'escape-room', 'name': ..."
14996,14996,104515,"Bowser Jr, Larry & Dottie",bowser-jr-larry-dottie,4,https://media.rawg.io/media/screenshots/eb8/eb...,"[{'id': 256822, 'slug': 'shark-dash', 'name': ..."
14997,14997,104020,Alkis Hadjinicolaou,alkis-hadjinicolaou,3,https://media.rawg.io/media/screenshots/0b0/0b...,"[{'id': 22450, 'slug': 'the-language-game', 'n..."
14998,14998,102714,ChaosForge,chaosforge,1,https://media.rawg.io/media/screenshots/c86/c8...,"[{'id': 250011, 'slug': 'jupiter-hell', 'name'..."


In [74]:
developers['games'] = developers['games'].apply(remove_string)
developers

SyntaxError: unterminated string literal (detected at line 1) (<unknown>, line 1)

In [46]:
l = []
for i in range(len(developers)):
    df = pd.json_normalize(developers['games'][i])
    df['company'] = developers['name'][i]
    l.append(df)

developer_df = pd.concat(l).reset_index()
developer_df

NotImplementedError: 